In [ ]:
import urllib
from requests import get
from bs4 import BeautifulSoup as Soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
chrome_options = Options()
# chrome_options.add_argument('--headless')

num_pages = 5

search_pages = []
for i in range(num_pages):
    search_pages.append(f'https://www.metmuseum.org/art/collection/search#!?material=Handkerchiefs' \
                 f'&offset={i*80}&perPage=80&pageSize=0&sortBy=Relevance&sortOrder=asc' \
                 f'&searchField=All&showOnly=openAccess')


def get_img(url: str):
    """get text descriptions from url"""
    hdr = {'User-Agent': 'Chrome/84.0.4103.106',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}

    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = Soup(html)
    for i in soup.findAll('img', {'class': 'result-card__image--art'}):
        image_name = i.attrs['src'].rsplit('/', 1)[-1]
        image_url = i.attrs['src'].replace('mobile-large', 'original')
        with open(f'assets/met/{image_name}', "wb") as f:
            f.write(get(image_url).content)

for i in search_pages:
    get_img(i)

In [ ]:
num_pages = 20
start_page = 10

search_pages = []

for i in range(num_pages):
    search_pages.append(f'https://collections.mfa.org/search/objects/*/handkerchief/images?page={start_page+i}')

def get_img(url: str):
    """get text descriptions from url"""
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8',
         'Connection': 'keep-alive'}

    d = get(url, headers=hdr)
    soup = Soup(d.content, 'html.parser')
    domain = 'https://collections.mfa.org'
    for foo in soup.findAll('div', {'class': 'primaryMedia img-wrap'}):
        page = foo.find('a').attrs['href']
        url = f'{domain}{page}'
        item_id = page.split("/")[2]
        print(item_id)
        download = f'{domain}/download/{item_id}'
        d = get(download, headers=hdr)
        soup2 = Soup(d.content, 'html.parser')
        links = soup2.findAll('a', {'target': '_blank'})
        if links:
            dl_link = links[1].attrs['href']
            with open(f'assets/mfa/{item_id}.jpg', "wb") as f:
                f.write(get(dl_link).content)

for i in search_pages:
    get_img(i)